In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import torch
import pickle
from sentence_transformers.util import semantic_search
import json 


/home/shagoto/codes/headless/test_repos/semantic_search_experiment/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
retriever_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [3]:
with open('./data/data_v4.json', 'r') as f:
  data = json.load(f)

In [4]:
data[0]

{'context': ' ১৯৭০ সালের ১২ নভেম্বর পূর্ব-বাংলায় ভয়াবহ ঝড় এবং জলোচ্ছ্বাসে ১০ লাখ মানুষ মারা যায়। বঙ্গবন্ধু তার নির্বাচনি প্রচারণা স্থগিত করে ত্রাণ কাজে ঝাঁপিয়ে পড়েন।',
 'qas': [{'qtns': ['পূর্ব বাংলায় ১৯৭০ এর ঝড়ে কত মানুষ মারা যায় ?',
    '১৯৭০ এর ঝড়ে কত মানুষ বাংলাদেশ মারা যায়?',
    'কত মানুষ ১৯৭০ এ মারা যায় ?',
    nan],
   'exact_answer': '১০ লাখ মানুষ ',
   'answer': '১৯৭০ এর ঝড়ে ১০ লাখ মানুষ মারা যায় এই বাংলায় ।',
   'tags': [nan, 'ইতিহাস'],
   'date': '11-12-1970',
   'year': 1970.0,
   'location': [nan, 'পূর্ব বাংলা'],
   'people': [nan, nan, nan],
   'event': 'জলোচ্ছ্বাস',
   'mujib_relavant': 'NO'},
  {'qtns': ['১৯৭০ এ শেখ মুজিব নির্বাচনী প্রচারনা বন্ধ করেন কেন?',
    '১৯৭০ এ বঙ্গবন্ধু নির্বাচনী প্রচরনা বন্ধ করেন কেন?',
    ' ',
    ' '],
   'exact_answer': 'প্রাকৃতিক দুর্যোগ এর জন্য ',
   'answer': 'বঙ্গবন্ধু তার নির্বাচনি প্রচারণা স্থগিত করেন প্রাকৃতিক দুর্যোগ এর জন্য  ',
   'tags': [nan, 'ইতিহাস'],
   'date': nan,
   'year': 1970.0,
   'location': [nan, 'পূর্ব বাংলা']

In [5]:
only_mujib = []
for item in data:
    for qas in item['qas']:
        if qas['mujib_relavant'] == 'YES':
            only_mujib.append(
                {
                    'context': item['context'],
                    'qas': qas
                }

            )

In [6]:
only_mujib[12]

{'context': '১৯৫৪  সালের মার্চ মাসে প্রদেশিক পরিষদের সাধারণ নির্বাচনে ২৩৭টি মুসলিম আসনের মধ্যে তৎকালীন আওয়ামী মুসলিম লীগ নেতৃত্বাধীন যুক্তফ্রন্ট ২২৩টি আসনে বিজয়ী হয়। আওয়ামী লীগ বিজয়ী হয় ১৪৫টি আসনে। পাকিস্তান মুসলিম লীগের ধর্মব্যবসায়ীদের প্রত্যাখান করে বাংলার জনগণ। মাত্র ৯টি আসন পেয়ে ধুয়ে মুছে সাফ হয়ে যায় মুসলিম লীগ। বঙ্গবন্ধু মুসলিম লীগের প্রভাবশালী এক নেতাকে ১৩ হাজার ভোটের বিশাল ব্যবধানে পরাজিত করেন। দেশজুড়ে প্রচার-প্রচারণা, অক্লান্ত শ্রম এবং তেজোদীপ্ত নেতৃত্বের কারণে আওয়ামী লীগের প্রধান মুখপাত্রে পরিণত হয় বঙ্গবন্ধু। জনগণের কাছে ন্যায্য অধিকার আদায় এবং শোষণের বিরুদ্ধে আন্দোলনের প্রাণপুরুষ হয়ে ওঠেন তিনি। শপথ নেন শেরে বাংলার নেতৃত্বাধীন যুক্তফ্রন্ট মন্ত্রিসভার কনিষ্ঠ মন্ত্রী হিসেবে। কৃষি, বন ও সমবায় মন্ত্রণালয়ের দায়িত্ব পান তিনি। কিন্তু পাকিস্তানের কেন্দ্রীয় সরকার ষড়যন্ত্র করে এই মন্ত্রিসভাকে পদচ্যুত এবং গ্রেফতার করে শেখ মুজিবুর রহমানকে।  ',
 'qas': {'qtns': ['যুক্তফ্রন্ট কার নেতৃত্বাধীন ছিল?',
   'যুক্তফ্রন্ট এর নেতৃত্ব দিয়েছিল কে? ',
   ' কার নেতৃত্বে যুক্তফ্রন্ট জয় লাভ করে ?',


In [7]:
questions = []
for item in only_mujib:
    # print(item)
    for qts in item['qas']['qtns']:
        # print(qts)

        if type(qts) == str and qts != ' ':
            questions.append(qts)

In [8]:
questions

['১৯৭০ এ শেখ মুজিব নির্বাচনী প্রচারনা বন্ধ করেন কেন?',
 '১৯৭০ এ বঙ্গবন্ধু নির্বাচনী প্রচরনা বন্ধ করেন কেন?',
 " কত সালে 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ এর অধিবেশনে দলের সাধারন সম্পাদক হিসেবে নির্বাচিত হন বঙ্গবন্ধু শেখ মুজিবুর রহমান ?",
 " বঙ্গবন্ধু শেখ মুজিবুর রহমান কত সালে 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ এর অধিবেশনে দলের সাধারন সম্পাদক হিসেবে নির্বাচিত হন  ?",
 " বঙ্গবন্ধু  'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ এর অধিবেশনে দলের সাধারন সম্পাদক হিসেবে নির্বাচিত হন কত সালে  ?",
 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগের অধিবেশন কতসালে অনুষ্ঠিত হয় ',
 'পূর্ব পাকিস্তান মুসলিম লীগের অধিবেশন কতসালে অনুষ্ঠিত হয় ',
 '১৯৫৩ সালে বঙ্গবন্ধু কোন দলের সাধারন সম্পাদক নির্বাচিত হন?',
 'শেখ মজীব  কোন দলের সাধারন সম্পাদক নির্বাচিত হন?',
 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ দলের কোন পদে নির্বাচিত হন শেখ মুজিবুর রহমান?',
 'শেখ মুজিবুর রহমান পূর্ব পাকিস্তান মুসলিম লীগ দলের কোন পদে নির্বাচিত হন ?',
 ' মুসলিম লীগ দলের কোন পদে শেখ মজীব ছিলেন  ?',
 ' দেশজুড়ে যুক্তফ্রন্ট প্রচারনা শুরু করেন কে?',
 ' কে যুক্

In [9]:
len(questions)

669

In [10]:
only_mujib[0]['qas']

{'qtns': ['১৯৭০ এ শেখ মুজিব নির্বাচনী প্রচারনা বন্ধ করেন কেন?',
  '১৯৭০ এ বঙ্গবন্ধু নির্বাচনী প্রচরনা বন্ধ করেন কেন?',
  ' ',
  ' '],
 'exact_answer': 'প্রাকৃতিক দুর্যোগ এর জন্য ',
 'answer': 'বঙ্গবন্ধু তার নির্বাচনি প্রচারণা স্থগিত করেন প্রাকৃতিক দুর্যোগ এর জন্য  ',
 'tags': [nan, 'ইতিহাস'],
 'date': nan,
 'year': 1970.0,
 'location': [nan, 'পূর্ব বাংলা'],
 'people': [nan, nan, nan],
 'event': 'জলোচ্ছ্বাস',
 'mujib_relavant': 'YES'}

In [11]:
only_mujib[0]['qas']['qtns'][0]

'১৯৭০ এ শেখ মুজিব নির্বাচনী প্রচারনা বন্ধ করেন কেন?'

In [12]:
questions = []
for item in data:
  for qas in item['qas']:
    for qts in qas['qtns']:
        if type(qts) == str:
            questions.append(qts)

## From Excel Sheet 

In [16]:
import pandas as pd
df = pd.read_excel("data/final_data.xlsx")
df.columns

Index(['Unnamed: 0', 'Context No', 'Context', 'ContextQuesNo', 'Q1', 'Q2',
       'Q3', 'Q4', 'Exact Answer', 'Answer', 'T1', 'T2', 'L1', 'L2', 'P1',
       'P2', 'P3', 'Event', 'Date', 'Year', 'Mujib Relevant'],
      dtype='object')

In [17]:
print(f"Length of the dataframe : {len(df)}") #including header

Length of the dataframe : 422


In [20]:
ques_df = df[['Q1','Q2','Q3','Q4','Mujib Relevant']]
ques_df.head()

,Q1,Q2,Q3,Q4,Mujib Relevant
0,পূর্ব বাংলায় ১৯৭০ এর ঝড়ে কত মানুষ মারা যায় ?,১৯৭০ এর ঝড়ে কত মানুষ বাংলাদেশ মারা যায়?,কত মানুষ ১৯৭০ এ মারা যায় ?,NaN,NO
1,১৯৭০ এ শেখ মুজিব নির্বাচনী প্রচারনা বন্ধ করেন ...,১৯৭০ এ বঙ্গবন্ধু নির্বাচনী প্রচরনা বন্ধ করেন কেন?,,,YES
2,কেন ১৯৭০ সালে ১০ লাখ মানুষ মারা যায় ?,NaN,NaN,NaN,NO
3,কত সালে 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ এ...,বঙ্গবন্ধু শেখ মুজিবুর রহমান কত সালে 'পূর্ব পা...,বঙ্গবন্ধু 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লী...,NaN,YES
4,পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগের অধিবেশন ক...,পূর্ব পাকিস্তান মুসলিম লীগের অধিবেশন কতসালে অন...,NaN,NaN,YES


In [21]:
mujib_relevant_ques = ques_df[ques_df['Mujib Relevant'] == "YES"]
mujib_relevant_ques

,Q1,Q2,Q3,Q4,Mujib Relevant
1,১৯৭০ এ শেখ মুজিব নির্বাচনী প্রচারনা বন্ধ করেন ...,১৯৭০ এ বঙ্গবন্ধু নির্বাচনী প্রচরনা বন্ধ করেন কেন?,,,YES
3,কত সালে 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ এ...,বঙ্গবন্ধু শেখ মুজিবুর রহমান কত সালে 'পূর্ব পা...,বঙ্গবন্ধু 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লী...,NaN,YES
4,পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগের অধিবেশন ক...,পূর্ব পাকিস্তান মুসলিম লীগের অধিবেশন কতসালে অন...,NaN,NaN,YES
5,১৯৫৩ সালে বঙ্গবন্ধু কোন দলের সাধারন সম্পাদক নি...,শেখ মজীব কোন দলের সাধারন সম্পাদক নির্বাচিত হন?,NaN,NaN,YES
8,পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ দলের কোন পদ...,শেখ মুজিবুর রহমান পূর্ব পাকিস্তান মুসলিম লীগ দ...,মুসলিম লীগ দলের কোন পদে শেখ মজীব ছিলেন ?,NaN,YES
...,...,...,...,...,...
417,শেখ মুজিবুর রহমানের কয় ছেলে ছিলো?,মুজিবের কতজন পুত্র সন্তান ছিলো?,NaN,NaN,YES
418,শেখ মুজিবুর রহমানের কয় মেয়ে ছিলো?,মুজিবের কতজন কন্যা সন্তান ছিলো?,NaN,NaN,YES
419,শেখ মুজিবুর রহমানের ছেলেমেয়ের নাম কি?,শেখ মুজিবের সন্তানদের নাম কি?,শেখ মুজিবের ছেলে মেয়ে কে কে?,NaN,YES
420,শেখ মুজিব রহমানের ছেলেদের নাম কি?,শেখ মুজিবের পুত্র সন্তানদের নাম কি?,শেখ মুজিবের পুত্র কে কে?,NaN,YES


In [24]:
q1 = mujib_relevant_ques['Q1'].dropna()


In [25]:
len(q1)

265

In [28]:
q1.head(3)

1    ১৯৭০ এ শেখ মুজিব নির্বাচনী প্রচারনা বন্ধ করেন ...
3     কত সালে 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ এ...
4    পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগের অধিবেশন ক...
Name: Q1, dtype: object

In [29]:
q2 = mujib_relevant_ques['Q2'].dropna()
q3 = mujib_relevant_ques['Q3'].dropna()
q4 = mujib_relevant_ques['Q4'].dropna()

In [31]:
print(f"Total Q1 : {len(q1)}")
print(f"Total Q2 : {len(q2)}")
print(f"Total Q3 : {len(q3)}")
print(f"Total Q4 : {len(q4)}")

Total Q1 : 265
Total Q2 : 244
Total Q3 : 127
Total Q4 : 35


In [56]:
q2_ques= []

for q in q2:
    q2_ques.append(q)

q2_idx = q2.index.tolist()

len(q2_ques) , len(q2_idx)

(244, 244)

In [57]:
q1_ques= []

for q in q1:
    q1_ques.append(q)

q1_idx = q1.index.tolist()

len(q1_ques) , len(q1_idx)

(265, 265)

In [58]:
q3_ques= []

for q in q3:
    q3_ques.append(q)

q3_idx = q3.index.tolist()

len(q3_ques) , len(q3_idx)

(127, 127)

In [59]:
q4_ques= []

for q in q4:
    q4_ques.append(q)

q4_idx = q4.index.tolist()

len(q4_ques) , len(q4_idx)

(35, 35)

In [61]:
for idx in q2_idx:
    for q1idx in q1_idx:
        if idx == q1_idx:
            q1.loc[idx]

In [65]:
q2_ques

['১৯৭০ এ বঙ্গবন্ধু নির্বাচনী প্রচরনা বন্ধ করেন কেন?',
 " বঙ্গবন্ধু শেখ মুজিবুর রহমান কত সালে 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ এর অধিবেশনে দলের সাধারন সম্পাদক হিসেবে নির্বাচিত হন  ?",
 'পূর্ব পাকিস্তান মুসলিম লীগের অধিবেশন কতসালে অনুষ্ঠিত হয় ',
 'শেখ মজীব  কোন দলের সাধারন সম্পাদক নির্বাচিত হন?',
 'শেখ মুজিবুর রহমান পূর্ব পাকিস্তান মুসলিম লীগ দলের কোন পদে নির্বাচিত হন ?',
 ' কে যুক্তফ্রন্টের প্রচারনা শুরু করে?',
 'আওয়ামী লীগের প্রধান  কে ছিলেন?',
 'যুক্তফ্রন্ট মন্ত্রীসভার কম বয়সী মন্ত্রী হিসেবে শপথ নেন কে?',
 'স্বাধীনতা আন্দোলনের প্রাধান পুরুষ কে?',
 ' কাকে কৃষি, বন ও সমবায় মন্ত্রণালয়ের দায়িত্ব দেওয়া হয়?',
 'মুসলীম লীগের প্রভাবশালী নেতাকে  পরাজিত করে কে?',
 ' মন্ত্রীসভাকে ভাঙার পরে  কাকে গ্রেফতার করা হয় ?',
 'যুক্তফ্রন্ট এর নেতৃত্ব দিয়েছিল কে? ',
 ' প্রদেশিক পরিষদের সাধারণ নির্বাচনে  দল বিজয়রে পরে শেখ মুজিব কোন মন্ত্রণালয়ের দায়িত্ব পান ?',
 'শেখ মুজিব মুসলীম লীগের  নেতাকে কত  ভোটের ব্যবধানে পরাজিত করেন?',
 'কবে শেখ মুজীব গণপরিষদের সদস্য হন ? ',
 'এঈ দেশের নাম পূর্ব পাকিস্তান ক

In [66]:
q2_embds = retriever_model.encode(q2_ques)

In [67]:
q2_embds.shape

(244, 384)

In [68]:
len(q2_ques)

244

In [69]:
q1_embds = retriever_model.encode(q1_ques)

In [70]:
q1_embds.shape

(265, 384)

In [89]:
found_idx= []
for q in q2_ques:
    q_emb = retriever_model.encode(q)
    query_embeddings = torch.FloatTensor(q_emb)
    hits = semantic_search(query_embeddings, q1_embds, top_k=1)
    
    # print(hits[0][0]['corpus_id'])
    found_idx.append(hits[0][0]['corpus_id'])
    


In [107]:
f_d = []
for i in range(len(found_idx)):
    f = found_idx[i]
    # if f <= q2_idx[-1]:
    f_d.append(q1_idx[found_idx[i]])
        


In [109]:
len(f_d)

244

In [110]:
f_d

[44,
 3,
 4,
 5,
 4,
 26,
 20,
 21,
 22,
 23,
 52,
 25,
 26,
 314,
 24,
 274,
 31,
 32,
 164,
 378,
 38,
 42,
 43,
 375,
 47,
 87,
 400,
 51,
 50,
 53,
 197,
 50,
 56,
 59,
 38,
 368,
 67,
 312,
 25,
 75,
 368,
 142,
 197,
 88,
 89,
 308,
 333,
 247,
 102,
 102,
 378,
 38,
 67,
 116,
 30,
 276,
 125,
 137,
 247,
 134,
 143,
 420,
 137,
 138,
 365,
 140,
 141,
 142,
 143,
 348,
 374,
 345,
 153,
 308,
 155,
 137,
 213,
 162,
 379,
 164,
 165,
 166,
 238,
 30,
 386,
 348,
 185,
 196,
 282,
 32,
 348,
 340,
 374,
 137,
 196,
 374,
 374,
 199,
 227,
 388,
 347,
 347,
 162,
 179,
 399,
 308,
 238,
 220,
 21,
 227,
 226,
 231,
 232,
 374,
 374,
 156,
 264,
 213,
 246,
 419,
 266,
 248,
 212,
 143,
 316,
 388,
 254,
 137,
 265,
 266,
 25,
 206,
 271,
 320,
 407,
 274,
 372,
 276,
 264,
 344,
 206,
 280,
 247,
 282,
 22,
 201,
 234,
 286,
 238,
 143,
 316,
 316,
 292,
 87,
 374,
 375,
 296,
 300,
 268,
 304,
 275,
 304,
 5,
 275,
 311,
 312,
 374,
 314,
 288,
 308,
 264,
 401,
 322,
 311,
 324

In [111]:
q2_idx

[1,
 3,
 4,
 5,
 8,
 14,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 38,
 42,
 43,
 44,
 47,
 48,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 59,
 61,
 66,
 67,
 71,
 74,
 75,
 76,
 85,
 87,
 88,
 89,
 90,
 93,
 95,
 102,
 103,
 105,
 114,
 116,
 117,
 123,
 124,
 125,
 126,
 132,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 145,
 146,
 148,
 153,
 154,
 155,
 157,
 161,
 162,
 163,
 164,
 165,
 166,
 168,
 169,
 170,
 173,
 178,
 179,
 180,
 185,
 189,
 190,
 191,
 193,
 196,
 197,
 198,
 199,
 201,
 206,
 210,
 211,
 213,
 215,
 216,
 217,
 219,
 220,
 221,
 222,
 226,
 228,
 229,
 231,
 232,
 238,
 240,
 241,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 264,
 265,
 266,
 267,
 268,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 292,
 293,
 296,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 308,
 311,
 312,
 313,
 314,
 315,
 316,
 320,
 321,
 322,
 323,
 324,
 32

In [116]:
count = 0
for i in range(len(f_d)):
    if f_d[i] == q2_idx[i]:
        count += 1
        
print("Total match Found : ", count)

model_accuracy =count/len(q2_idx)
print("Models Accuracy : ",model_accuracy)
    

Total match Found :  78
Models Accuracy :  0.319672131147541


In [105]:
q2_idx[-1]

421

In [94]:

len(found_idx), len(q2_idx)

count = 0
for i in range(len(found_idx)):
    if found_idx[i] == q2_idx[i]:
        count += 1

print(count)
        

0


In [103]:
count = 0
for i in range(len(found_idx)):
    for q2idx in q2_idx:
    
        if found_idx[i] == q2idx:
            count += 1

print(count)

112


In [101]:
count = 0
for i in range(len(found_idx)):
    for q1idx in q1_idx:
        
        if found_idx[i] == q1idx:
            count += 1

print(count)

119


In [102]:
count = 0
for i in range(len(q2_idx)):
    for q1idx in q1_idx:
        
        if q2_idx[i] == q1idx:
            count += 1

print(count)

244


In [95]:
q2_ques[1]

" বঙ্গবন্ধু শেখ মুজিবুর রহমান কত সালে 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ এর অধিবেশনে দলের সাধারন সম্পাদক হিসেবে নির্বাচিত হন  ?"

In [73]:
q1_ques[23]

' কত সালে বঙ্গবন্ধু মন্ত্রিসভা থেকে স্বেচ্ছায় পদত্যাগ করেন?'

In [98]:
q2_ques[3]

" বঙ্গবন্ধু শেখ মুজিবুর রহমান কত সালে 'পূর্ব পাকিস্তান আওয়ামী মুসলিম লীগ এর অধিবেশনে দলের সাধারন সম্পাদক হিসেবে নির্বাচিত হন  ?"

In [100]:
q1_ques[3]

'১৯৫৩ সালে বঙ্গবন্ধু কোন দলের সাধারন সম্পাদক নির্বাচিত হন?'

# Main Methods

In [117]:
found_idx= []
for q in q3_ques:
    q_emb = retriever_model.encode(q)
    query_embeddings = torch.FloatTensor(q_emb)
    hits = semantic_search(query_embeddings, q1_embds, top_k=1)
    
    # print(hits[0][0]['corpus_id'])
    found_idx.append(hits[0][0]['corpus_id'])
    

In [118]:
f_d = []
for i in range(len(found_idx)):
    f = found_idx[i]
    # if f <= q2_idx[-1]:
    f_d.append(q1_idx[found_idx[i]])
        

In [120]:
count = 0
for i in range(len(f_d)):
    if f_d[i] == q3_idx[i]:
        count += 1
        
print("Total match Found : ", count)

model_accuracy =count/len(q3_idx)
print("Models Accuracy : ",model_accuracy)
    

Total match Found :  37
Models Accuracy :  0.29133858267716534


## Test 

In [121]:
found_idx= []
for q in q4_ques:
    q_emb = retriever_model.encode(q)
    query_embeddings = torch.FloatTensor(q_emb)
    hits = semantic_search(query_embeddings, q1_embds, top_k=1)
    
    # print(hits[0][0]['corpus_id'])
    found_idx.append(hits[0][0]['corpus_id'])
    

In [122]:
f_d = []
for i in range(len(found_idx)):
    f = found_idx[i]
    # if f <= q2_idx[-1]:
    f_d.append(q1_idx[found_idx[i]])

In [125]:
len(f_d) , len(q4)

(35, 35)

In [126]:
count = 0
for i in range(len(f_d)):
    if f_d[i] == q4_idx[i]:
        count += 1
        
print("Total match Found : ", count)

model_accuracy =count/len(q4_idx)
print("Models Accuracy : ",model_accuracy)

Total match Found :  13
Models Accuracy :  0.37142857142857144
